# Import Libraries

In [33]:
################ WEB SCRAPING MODULES ############
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import bs4
from fake_useragent import UserAgent
import requests
################ TIME MODLULES ###################
import time
from datetime import date 
import datetime
############## DATA MANIPULATION MODULES #########
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords

# Read 100 questions list

In [34]:
df = pd.read_excel('AnswersFromSource11_20.xlsx',engine='openpyxl')
link = 'https://www.autismsa.org.au/frequently-asked-questions-about-asd'
df[link]=np.nan
#df.drop(df.iloc[:, 1:25], inplace = True, axis = 1)
df

,Question,https://springbrookbehavioral.com/frequently-asked-questions-about-autism/,https://www.aacap.org/AACAP/Families_and_Youth/Resource_Centers/Autism_Resource_Center/FAQ.aspx,https://www.autism.org.sg/living-with-autism/myths-and-facts-of-autism,https://www.autismsa.org.au/frequently-asked-questions-about-asd
0,What Can I Do To Prepare My Child For The Future?,Early intervention can do a lot toward fosteri...,NaN,NaN,NaN
1,Did I Cause My Child’s Autism?,"No. After getting an ASD diagnosis, this is th...",NaN,NaN,NaN
2,Can My Child’s Autism Be Cured?,No. Autism is not a disease that can be cured—...,NaN,NaN,NaN
3,How Can I Deal With This Diagnosis?,"Start by being patient and kind with yourself,...",NaN,NaN,NaN
4,How Do I Find Help For My Child?,Start by making sure that you and your child h...,NaN,NaN,NaN
...,...,...,...,...,...
109,What are some ways that parents can reduce the...,NaN,NaN,NaN,NaN
110,Do some families deal with stress better than ...,NaN,NaN,NaN,NaN
111,Do siblings suffer increased stress as a resul...,NaN,NaN,NaN,NaN
112,What can I do about my children’s stress?,NaN,NaN,NaN,NaN


# Scrape QA pairs from website

In [35]:
# Open webpage in a new window
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get('https://www.autismsa.org.au/frequently-asked-questions-about-asd')



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [/home/gopan/.wdm/drivers/chromedriver/linux64/92.0.4515.107/chromedriver] found in cache


In [36]:
# Parse text in webpage
source = driver.page_source
soup = bs4.BeautifulSoup(source, 'html.parser')

In [37]:
# Search the questions mentioned in webpage
l = []
for i in soup.find_all('b'):
    print(i.text)
    l.append(i.text)

What causes ASD?
How common is ASD?
Do all individuals with ASD have intellectual disability?
How do individuals with ASD communicate?
How does ASD affect a person during their life?


In [38]:
# Tried to search for answers of questions
k=[]
for p in soup.find_all('p'):
    if p.find('b'):
        continue
    k.append(p.text)
for li in k:
    print(li)
    print("\n")
    print("\n")   

 The cause is unknown at present but it is generally accepted that it is caused by variations in brain structure or function. There is evidence to suggest that genetics plays an important role in the cause of ASD; it is likely to be multiple genes. Genes may create a “susceptibility” to developing ASD but researchers have not yet identified a single "trigger" that causes ASD to develop. There is no medical or genetic screening currently available.




ASD affects 1 in 100 individuals (Fernell and Gillberg, 2010) in Australia and more males than females. The male to female ratio is approximately 4:1 (Freitag, 2007; Abrahams and Geschwind, 2008)




Some individuals with ASD have an intellectual disability whilst some have average or above average intelligence. Often individuals with ASD show an uneven pattern of skills having areas of particular strength and other areas where support may be required.




Individuals with ASD vary greatly in their communication skills. Some individuals a

# Check which questions are similar

In [39]:
stop_words = set(stopwords.words('english'))

def clean_text(sent):
    sent = sent.lower() # lowercase
    sent = re.sub(r'[^\w\s]', '', sent) # remove punctuations
    sent = re.sub('Autism Spectrum Disorder','ASD',sent) # Compress term
    sent = [w for w in sent.split() if not w.lower() in stop_words] # Remove stopwords
    sent = " ".join(sent)
    return sent

In [40]:
def jaccard_similarity(list1, list2):
    s1 = set(list1)
    s2 = set(list2)
    return float(len(s1.intersection(s2)) / len(s1.union(s2)))

In [41]:
for i in l:
    i2 = clean_text(i)
    list1 = i2.split()
    
    temp1 = 0.0
    temp2 = ''
    temp3 = ''
    
    for j in df['Question']:
        j2 = clean_text(j)
        list2 = j2.split()
            
        sim = jaccard_similarity(list1, list2)
        
        if(sim>temp1):
            temp1 = sim
            temp2 = j
            temp3 = i
            
    if(temp1>=0.3):
        print('Website --> ',temp3)
        print('100 questions list --> ',temp2)
        print(temp1)
        print('*'*100)

In [42]:
# Directly add answer to 'What are the Autism Spectrum Disorders (ASD)?' in existing dataframe
#df.iloc[2,0]
#df.iloc[1,1]=k[3]

In [43]:
l

['What causes ASD?',
 'How common is ASD?',
 'Do all individuals with ASD have intellectual disability?',
 'How do individuals with ASD communicate?',
 'How does ASD affect a person during their life?']

In [44]:
#l=l[1:]
#l

In [45]:
# manually select answers to questions in list 'l'

#l2 = [k[8]+'\n'+k[12]+'\n'+k[13]+'\n'+k[14],
#      k[16]+'\n'+k[17],
#     k[19]+'\n'+k[21]]
l2 = [k[0],k[1],k[2],k[3],k[4]+"\n"+k[5]]

In [46]:
# Create new dataframe with QA pairs
df2 = pd.DataFrame(zip(l,l2),columns=['Question',link])
df2

,Question,https://www.autismsa.org.au/frequently-asked-questions-about-asd
0,What causes ASD?,The cause is unknown at present but it is gen...
1,How common is ASD?,ASD affects 1 in 100 individuals (Fernell and ...
2,Do all individuals with ASD have intellectual ...,Some individuals with ASD have an intellectual...
3,How do individuals with ASD communicate?,Individuals with ASD vary greatly in their com...
4,How does ASD affect a person during their life?,ASD is a lifelong condition. Most individuals ...


In [47]:
# Concatenate existing and new dataframes
df3 = pd.concat([df,df2],axis=0)
df3 = df3.sort_values(by=list(df3.columns[1:])).reset_index(drop=True)
df3

,Question,https://springbrookbehavioral.com/frequently-asked-questions-about-autism/,https://www.aacap.org/AACAP/Families_and_Youth/Resource_Centers/Autism_Resource_Center/FAQ.aspx,https://www.autism.org.sg/living-with-autism/myths-and-facts-of-autism,https://www.autismsa.org.au/frequently-asked-questions-about-asd
0,What Can I Do To Prepare My Child For The Future?,Early intervention can do a lot toward fosteri...,NaN,NaN,NaN
1,Did I Cause My Child’s Autism?,"No. After getting an ASD diagnosis, this is th...",NaN,NaN,NaN
2,Can My Child’s Autism Be Cured?,No. Autism is not a disease that can be cured—...,NaN,NaN,NaN
3,How Can I Deal With This Diagnosis?,"Start by being patient and kind with yourself,...",NaN,NaN,NaN
4,How Do I Find Help For My Child?,Start by making sure that you and your child h...,NaN,NaN,NaN
...,...,...,...,...,...
114,What are some ways that parents can reduce the...,NaN,NaN,NaN,NaN
115,Do some families deal with stress better than ...,NaN,NaN,NaN,NaN
116,Do siblings suffer increased stress as a resul...,NaN,NaN,NaN,NaN
117,What can I do about my children’s stress?,NaN,NaN,NaN,NaN


# Save Output

In [48]:
df3.to_excel('AnswersFromSource11_20.xlsx',index=False)